In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd

#my helper functions from ipynb file
from ipynb.fs.full.PostgreHelper import * 

# Extract

In [18]:
#my sample google sheet
sheet_id = '1xo1_DZEVBiVz39S5Ln3CSnagbPE78cvBDKtwqtC3hKI'

#extract all tables from the sheet
student_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Students"
df_student = pd.read_csv(student_url)

st_course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Student_Courses"
df_st_course = pd.read_csv(st_course_url)

course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Courses_Info"
df_course_info = pd.read_csv(course_url)

payment_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Payment_Method"
df_payment = pd.read_csv(payment_url)

purchased_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Purchased_Info"
df_purchase = pd.read_csv(purchased_url)

In [20]:
df_payment

,payment_method_id,student_id,payment_type,account_no,active
0,P001,S001,visa,1,yes
1,P002,S002,paypal,2,yes
2,P003,S003,master,3,yes
3,P004,S004,visa,4,yes
4,P005,S005,paypal,5,yes
5,P006,S006,master,6,yes
6,P007,S007,visa,7,yes
7,P008,S008,paypal,8,yes
8,P009,S009,master,9,yes
9,P010,S010,visa,10,yes


# Transform (Dimensional Model)

# Load

In [ ]:
#connect to my Postgre database that hosted on ElephantSql with free instance
connection, cursor = connect(host = 'arjuna.db.elephantsql.com', 
                             database = '*******', 
                             user = '*******', 
                             password = '*******', 
                             port = '5432')

In [ ]:
#create new table for the above youtube data
query = 'CREATE TABLE youtube_channel \
        (id SERIAL PRIMARY KEY, title text, url text, liked int, disliked int, views int, comment_count int); '
create_table(connection, cursor, query)

In [ ]:
#inserting the dataframe into youtube_channel table

#convert dataframe in tuples to iterate
tuples = [tuple(x) for x in df.to_numpy()]

#df columns into string
cols = ','.join(list(df.columns))

#insert query
query = "INSERT INTO youtube_channel(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s)" % (cols)

#inserting with executemany function
try:
    cursor.executemany(query, tuples)
    print("Data inserted using execute_many() successfully!")
except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    cursor.close()

In [ ]:
#read PostgreSQL table into pandas dataframe
data = pd.read_sql_query(con = connection, sql = 'SELECT * FROM youtube_channel')
data.head(10)